In [1]:
%ls

 fine_tune.py             myjob_4GPUs_412117.out   output_padded2/
 LICENSE                  myjob_4GPUs_412162.err   predict.py
 models/                  myjob_4GPUs_412162.out   README.md
 myjob2.slurm             myjob.slurm              tests/
 myjob_4GPUs_412113.err  'nodelist.$'              train.py
 myjob_4GPUs_412113.out   output/                  Untitled.ipynb
 myjob_4GPUs_412117.err   output_padded/           utils/


In [4]:
from tqdm import tqdm

In [2]:
import os
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import numpy as np
import torch.nn as nn
import torch.optim as optim
from utils.dataloader import get_dataloaders, save_label_to_idx, CustomDataset
from utils.model_utils import train_model, save_model, generate_unique_model_name
from models.model_factory import model_factory

/home/birdy/.conda/envs/cv-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## load up training, testing, and validation data

In [5]:
data_dir = "/home/birdy/meng_thesis/data/unsorted_MGL1704_data"
label_path = "/home/birdy/meng_thesis/data/mgl1704_labels.csv"
# Data preparation
dataloaders = get_dataloaders(data_dir, label_path, batch_size=16, my_transforms="PadToMaxSize")

Transforms being applied: [<utils.my_transforms.PadToMaxSize object at 0x2008b5439d20>, ToTensor()]


## load model architecture and pretrained weights 

In [6]:
num_classes = dataloaders['train'].dataset.num_classes()
input_size = dataloaders["train"].dataset[0][0].size()
model_name = "resnet"
model = model_factory(model_name, num_classes=num_classes, input_size=input_size)

In [7]:
pretrained_weights = "/home/birdy/meng_thesis/code/master_ifcb_classifier/output_padded2/resnet_20240619_071244_weights.pth"
model.load_state_dict(torch.load(pretrained_weights))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## run model on testing data

In [8]:
test_loader = dataloaders["test"]
criterion = torch.nn.CrossEntropyLoss()
running_loss = 0.0
correct = 0
total = 0
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(accuracy)

100%|██████████| 2072/2072 [09:35<00:00,  3.60it/s]

1.4875384708225212


PERFORMED HORRIBLY. What is the issue? 

## Try again with training data 

In [20]:
test_loader.dataset[0][0].size()

torch.Size([3, 1359, 890])

In [22]:
train_loader = dataloaders["train"]
criterion = torch.nn.CrossEntropyLoss()
running_loss = 0.0
correct = 0
total = 0
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for images, labels in tqdm(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(accuracy)

100%|██████████| 6240/6240 [26:13<00:00,  3.97it/s]

2.6983173076923075


### trying again with other validation method 

In [33]:
from sklearn.metrics import accuracy_score

In [31]:
def get_validation_results(model, dataloader):
    was_training = model.training
    model.eval()

    true_vals = []
    pred_vals = []
    with torch.no_grad():
        for i, (inputs, labels) in tqdm(enumerate(dataloader)):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            true_vals.append(labels)
            pred_vals.append(preds)
    return true_vals, pred_vals

In [34]:
true_vals, pred_vals = get_validation_results(model, dataloaders["test"])

2072it [08:11,  4.22it/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [47]:
true_val_conv = []
for t in true_vals:
    true_val_conv.extend(t.to("cpu"))
    
pred_val_conv = []
for t in pred_vals:
    pred_val_conv.extend(t.to("cpu"))

accuracy_score(true_val_conv, pred_val_conv)

0.014603825961016233

In [48]:
true_vals, pred_vals = get_validation_results(model, dataloaders["train"])
true_val_conv = []
for t in true_vals:
    true_val_conv.extend(t.to("cpu"))
    
pred_val_conv = []
for t in pred_vals:
    pred_val_conv.extend(t.to("cpu"))

accuracy_score(true_val_conv, pred_val_conv)

6240it [24:31,  4.24it/s]


0.02782451923076923